# Importation des Bibliothèques utiles

In [1]:
import  time

In [2]:
%matplotlib inline 
import numpy as np
import matplotlib.pyplot as plt
from random import *
from pylab import *
import unicodedata
from math import *
import re
import unicodedata

In [3]:
# Petit rappel pour avoir le temps d'éxécution d'une cellule
start_time = time.time()
# Insérer code entre ces deux lignes
print("Temps d execution : %s secondes ---" % (time.time() - start_time))

Temps d execution : 5.269050598144531e-05 secondes ---


In [4]:
#Banque6: 300K mots, dont on remplace les caractères spéciaux par des caractères de l'alphabet
#Les accents de cette banque sont mal lus par Python, quelle que soit la méthode utilisée.
#Nous avons donc décidé de les remplacer par des caractères réguliers.
#Par exemple, 'é' devient 'e'.
def supAccent(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
                  if unicodedata.category(c) != 'Mn')
file6=[]
with open(r"mots.txt", "rt", encoding='utf_8') as ligne:
    for lettre in ligne:
        lettre=supAccent(lettre)
        file6.append(lettre)




# Définition du dictionnaire de l'alphabet latin

In [5]:
################# ALPHABET LATIN #################

def alphabetLatin():
    dictionnaire={}
    for i in range(97,123): #lettres de a à z
        dictionnaire[chr(i)]=i-97
    for i in range(224,254):
        dictionnaire[chr(i)]=i-198 #toutes les lettres accentuées
    del(dictionnaire['÷']) #il y a ce caractère sépcial au milieu des lettres accentuées
    return dictionnaire

alphabetLatin=alphabetLatin()
#alphabetLatin[a] renvoie 0

def alphabetInverse(dictionnaire):
    inverse = {}
    for cle in dictionnaire:
        valeur = dictionnaire[cle]
        inverse[valeur]=cle
    return inverse

alphabetLatinInverse=alphabetInverse(alphabetLatin)
#alphabetLatinInverse[0] renvoie a

alphabet=alphabetLatin

# Banque de mots avec laquelle on travaille pour l'évaluation

In [6]:
#banques recommandées pour le français:
#file5 (140K mots, avec caractères accentués), file7 (400K mots, sans caractères accentués)
#utilise file6 (300K mots) pour l'évaluation des programmes
file = file6

#choisir l'alphabet avec lequel travailler:
alphabet = alphabetLatin
#c'est le seul disponible ici

################# ENLEVER \n A LA FIN DE LA BANQUE AVEC LAQUELLE ON TRAVAILLE #################

#Les mots de certaines listes contiennent un '\n' à la fin.
#On souhaite les supprimer pour que la taille d'une ligne soit égale à la taille de son mot dans la banque
#avec laquelle on travaille.
#Le programme retire également les caractères indisérés non présents dans notre alphabet.

def filtrage2(fichier,alphabet):
    nouvelleBanque=[]
    for mot in fichier:
        mot2=u''
        for lettre in mot:
            if lettre in alphabet:
                  mot2+=lettre
        if len(mot2)>0: #certaines lignes peuvent ne contenir qu'un caractère, et il peut ne pas être dans l'alphabet. 
            nouvelleBanque.append(mot2) #après filtrage, la ligne serait de longueur nulle. On ne la prend pas.
    return nouvelleBanque

file=filtrage2(file,alphabet)

# Génération de mots

# Méthodes sans Markov

In [7]:

################# FONCTIONS CREANT LES MATRICES DE COMPTAGE #################

#Cas particulier pour la génération de la première lettre du mot : tableau (matrice 1D) d'occurences de chaque lettre 
#comme première lettre d'un mot du fichier

def matriceComptage1(file,alphabet):
    comptage=np.zeros(55) #le tableau où sera stocké les occurences de chaque lettre comme première lettre d'un mot du fichier
    comptageTot=0 #comptage total des premières lettres de mots
    for ligne in file: #On parcourt chaque mot du fichier
        lettre=ligne[0] #On cherche sa première lettre
        i=alphabet[lettre] #On cherche le numéro i associé à cette lettre
        comptage[i]+=1 
        comptageTot+=1 
    return comptage, comptageTot
#comptage[0] par exemple, compte le nombre de fois où la lettre 'a' est en tête d'un mot dans le fichier.


#Matrice de comptage 55 x 55 des enchaînements de deux lettres des mots du fichier
def matriceComptage2(file,alphabet):
    P=np.zeros((55,55)) #Matrices des enchaîenements de deux lettres
    comptageTot=np.zeros(55) #Tableau du nombre d'enchainements rencontrés commençant par une lettre donnée, pour chaque lettre
    for ligne in file: #On parcourt les mots du fichier
        placeLettre=0 #Variable qui nous donne la place de la lettre qu'on analyse
        for lettre in ligne: #On parcourt chaque lettre du mot 
            i=alphabet[lettre] #Numéro associé à la lettre analysée
            if placeLettre<len(ligne)-1: #Si on n'est pas en fin de mot
                lettreSuivante=ligne[placeLettre+1] #On regarde la lettre suivante
                j=alphabet[lettreSuivante] #On cherche le numéro associé à la lettre suivante 
                P[i][j]+=1 #On ajoute 1 à la case de la matrice correspondant à l'enchainement lettre-lettreSuivante
                comptageTot[i]+=1 #On ajoute 1 au comptage du nombre d'enchainements rencontrés pour la lettre lettre
            placeLettre+=1
    return P, comptageTot



#Matrice de comptage 55 x 55 x 55 des enchaînements de trois lettres des mots du fichier
#Même principe que la matrice précédente
def matriceComptage3(file,alphabet):
    P=np.zeros((55,55,55))
    comptageTot=np.zeros((55,55))
    for ligne in file:
        placeLettre=0 #place de la lettre analysée dans le mot
        for lettre in ligne:
            i=alphabet[lettre]
            if placeLettre<len(ligne)-2: #On vérifie que la lettre analysée est bien suivie de deux autres lettres
                lettreSuivante=ligne[placeLettre+1]
                lettreApres=ligne[placeLettre+2]
                j=alphabet[lettreSuivante]
                k=alphabet[lettreApres]
                P[i][j][k]+=1
                comptageTot[i][j]+=1
            placeLettre+=1
    return P, comptageTot



#Matrice de comptage d'enchaînements de deux lettres au début des mots du fichier
#On cherche les occurences de tous les enchaînements de deux lettres lorsque l'enchaînement est en tête de mot
def matriceComptageDebut(file,alphabet):
    P=np.zeros((55,55))
    comptageTot=np.zeros(55)
    for ligne in file:
        if len(ligne)>=2: #On vérifie que le mot a plus de deux caractères
            lettre1=ligne[0]
            lettre2=ligne[1]
            i=alphabet[lettre1]
            j=alphabet[lettre2]
            P[i][j]+=1
            comptageTot[i]+=1
    return P, comptageTot



#Matrice de comptage des enchaînements de deux lettres à la fin des mots du fichier
#On cherche les occurences de tous les enchaînements de deux lettres, lorsque l'enchaînement est en fin de mot
def matriceComptageFin(file,alphabet):
    P=np.zeros((55,55))
    comptageTot=np.zeros(55)
    for ligne in file:
        if len(ligne)>=2:
            lettre1=ligne[len(ligne)-2]#avant-dernière lettre du mot analysé
            lettre2=ligne[len(ligne)-1] #dernière lettre du mot analysé
            i=alphabet[lettre1]
            j=alphabet[lettre2]
            P[i][j]+=1
            comptageTot[i]+=1
    return P, comptageTot




################# FONCTIONS CREANT LES MATRICES DE PROBABILITES #################

#fonction transformant un tableau de comptage (matrice 1D) en tableau de probabilités 
#(ne sert que pour la génération de la première lettre du mot)
def probas(tab, comptageTot):
    for i in range(len(tab)):
        tab[i]=tab[i]/comptageTot
    return tab


#fonction transformant une matrice de comptage 2D en matrice de probabilités
def matriceProbas(M, comptageTot):
    for i in range(len(comptageTot)):
        if comptageTot[i]!=0:
            M[i]=M[i]/comptageTot[i]
    return M

#fonction transformant une matrice de comptage 3D (pour les enchaînements de 3 lettres) en matrice de probabilités
def matriceProbas2(M, comptageTot):
    for i in range(len(M)):
        for j in range(len(M[i])):
            if comptageTot[i][j]!=0:
                M[i][j]=(M[i][j])/(comptageTot[i][j])
    return M


################# VARIABLES #################

#conversion de l'alphabet en liste pour pouvoir utiliser np.random.choice dessus (utile dans la génération de mot)
#on rappelle que l'aphabet est un dictionnaire python
def conversionAlphabet(alphabet):
    tab=[]
    for lettre in alphabet:
        tab.append(lettre)
    return tab

alphabetConverti=conversionAlphabet(alphabet)

#MATRICES DE COMPTAGE

comptage1, comptageTot1=matriceComptage1(file,alphabet)

P, comptageTot=matriceComptage2(file,alphabet)

PPrime, comptageTotPrime=matriceComptage3(file, alphabet)

PDebut, comptageTotDebut=matriceComptageDebut(file,alphabet)

PFin, comptageTotFin=matriceComptageFin(file,alphabet)

#MATRICES DE PROBABILITES

tabProbas1=probas(comptage1, comptageTot1) #tableau de probailités de chaque lettre en tant que première lettre de mot

P=matriceProbas(P,comptageTot) #matrice 2D de probabilités des enchaînements de deux lettres

PDebut=matriceProbas(PDebut, comptageTotDebut) #matrice 2D de probabilités des enchaîenments de deux lettres en début de mot

PFin=matriceProbas(PFin, comptageTotFin) #matrice 2D de probabilités des enchainements de deux lettres en fin de mot

PPrime=matriceProbas2(PPrime, comptageTotPrime) #matrice 3D de probabilités des enchainements de trois lettres

################### GENERATION DE MOTS #####################

#Génération de la première lettre du mot
def genererlettre1():
    res=np.random.choice(alphabetConverti,1,p=tabProbas1) #sélectionne 1 lettre dans alphabetConverti avec des 
    return res #probabilités données dans tabProbas1
#note: np.random.choice renvoit un tableau  

#Génération d'un mot de longueur n avec la matrice de probas des enchainements de deux lettres
#attention n>=1
def genererMot2(n,alphabet):
    tab=genererlettre1() #on chosit une première lettre
    lettre=tab[0] 
    mot=lettre #on initialise notre mot à créer avec cette première lettre
    for i in range(1,n):
        noLettre=alphabet[lettre] #numéro de la lettre 
        choix=np.random.choice(alphabetConverti,1,p=P[noLettre]) #choix de la lettre suivante en fonction de la lettre précédente
        lettreSuivante=choix[0]
        mot+=lettreSuivante #on ajoute cette nouvelle lettre au mot
        lettre=lettreSuivante #la nouvelle lettre devient 'lettre' et on réitère le processus avec cette nouvelle lettre
    return mot


#Génération de mots par enchainement de deux lettres avec contrôle des enchaînements en tête et queue de mot
#Meme principe que le programme précédent
#attention n>=3
def genererMot2bis(n,alphabet):
    tab=genererlettre1() #on choisit une première lettre
    lettre=tab[0]
    mot=lettre
    ind=alphabet[lettre] #numéro de la lettre 
    tab=np.random.choice(alphabetConverti,1,p=PDebut[ind]) #on choisit la lettre suivante en fonction de la lettre précédente
    lettre=tab[0] #sachant qu'elle est en début de mot
    mot+=lettre
    for i in range(2,n-1):
        noLettre=alphabet[lettre]
        choix=np.random.choice(alphabetConverti,1,p=P[noLettre])
        lettreSuivante=choix[0]
        mot+=lettreSuivante
        lettre=lettreSuivante
    ind=alphabet[lettre] #numéro de l'avant-dernière lettre
    tab=np.random.choice(alphabetConverti,1,p=PFin[ind]) #choix de la dernière lettre en fonction de l'avant-dernière lettre
    lettreFinale=tab[0] #sachant qu'elle est en fin de mot
    mot+=lettreFinale
    return mot


#Génération de mots par enchainement de trois lettres
#n>=1
#mots sortis de longueur 2n-1
def genererMot3(n,alphabet):
    tab=genererlettre1() #on génère d'abord les deux premières lettres du mot comme dans genererMot2bis
    lettre=tab[0]
    mot=lettre
    noLettre=alphabet[lettre]
    choix=np.random.choice(alphabetConverti,1,p=P[noLettre])
    lettreSuivante=choix[0]
    mot+=lettreSuivante
    lettrePrecedente, lettre=lettre, lettreSuivante
    for i in range(2,n):
        noLettre=alphabet[lettre]
        noLettrePrecedente=alphabet[lettrePrecedente]
        choix=np.random.choice(alphabetConverti,1,p=PPrime[noLettrePrecedente][noLettre]) #choix de la lettre suivante en
        lettreSuivante=choix[0] #fonction des deux lettres précédentes
        mot+=lettreSuivante
        lettrePrecedente, lettre=lettre, lettreSuivante #on recommence le processus
    return mot

#génération de mots par enchainement de trois lettres avec contrôle des enchaînements en tête et queue de mot
#n>=3
#mots sortis de longueur 2n-1
def genererMot3bis(n,alphabet):
    tab=genererlettre1() #on choisit une première lettre
    lettre=tab[0]
    mot=lettre
    noLettre=alphabet[lettre] 
    tab=np.random.choice(alphabetConverti,1,p=PDebut[noLettre]) #on choisit la lettre suivante en fonction de la 
    lettreSuivante=tab[0]#lettre précédente sachant qu'elle est en début de mot
    mot+=lettreSuivante
    lettrePrecedente, lettre=lettre, lettreSuivante
    for i in range(2,n-1):
        noLettre=alphabet[lettre]
        noLettrePrecedente=alphabet[lettrePrecedente]
        choix=np.random.choice(alphabetConverti,1,p=PPrime[noLettrePrecedente][noLettre]) #choix de la lettre suivante en
        lettreSuivante=choix[0] #fonction des deux lettres précédentes
        mot+=lettreSuivante
        lettrePrecedente, lettre=lettre, lettreSuivante #on recommence le processus
    noLettre=alphabet[lettre]
    tab2=np.random.choice(alphabetConverti,1,p=PFin[noLettre]) #on choisit la dernière lettre en fonction de l'avant-dernière
    lettreFinale=tab2[0] #sachant qu'on est en fin de mot
    mot+=lettreFinale
    return mot

##################### VISIONNER LES RESULTATS #####################

def vision(n,nombremots,alphabet,nomduprogramme):
    sum=0
    print("génération de mot avec"+str(nomduprogramme))
    for i in range(nombremots):
        t0=time.time()
        mot=nomduprogramme(n,alphabet)
        tf=time.time()
        sum+=(tf-t0)
        print(mot)
    print("temps moyen de génération d'un mot:" +str(sum/nombremots)+"secondes" )

vision(10,10,alphabet,genererMot3bis)

génération de mot avec<function genererMot3bis at 0x7f547cbfd290>
defrespant
frerefarch
arnessento
perojerota
bivassorce
fonsiezats
draciserra
aningetsai
cosemissas
fraissisat
temps moyen de génération d'un mot:0.0011120080947875977secondes


## Méthodes avec Markov

In [8]:
##################### MATRICES DE PROBABILITES P1, P2, ..., PplaceLettre, Pfin#####################

#création de la matrice de probabilité des enchainements de deux lettres lorsque la première lettre de
#l'enchainement est à la placeLettre-ième position d'un mot

def matriceProbaPosition1(file,placeLettre,alphabet):
    if placeLettre==0 : #cas particulier: tableau (matrice1D) de probas pour les lettres en 1ère place
        comptage1=np.zeros(55)
        comptageTot1=0
        for ligne in file: 
            l=ligne[0]  #on parcourt la première de chaque mot
            i=alphabet[l]
            comptage1[i]+=1 #on compte les différentes lettres 
            comptageTot1+=1 #on compte le nombre de lettres totales
        P=[comptage1[i]/comptageTot1 for i in range(0,55)] #normalisation
    else : 
        P=np.zeros((55,55))
        comptageTot=np.zeros(55)
        for ligne in file:
            if placeLettre<len(ligne)-1: 
                i=ligne[placeLettre-1] #lettre d'avant
                j=ligne[placeLettre] #lettre étudiée
                i=alphabet[i] #numéro associé à la lettre d'avant
                j=alphabet[j] #numéro associé à la lettre étudiée
                P[i][j]+=1 #on compte les lettres
                comptageTot[i]+=1 #on compte les lettres totales
        for i in range(len(comptageTot)):
            if comptageTot[i]!=0:
                P[i]=P[i]/comptageTot[i] #on convertit en probabilités
    return P


def matricesStockés(nombreDeLettres,file,alphabet): #cette fonction sert à ne pas recalculer les matrices pour chaque 
    Mstocks=[matriceProbaPosition1(file,i,alphabet) for i in range(nombreDeLettres-1)] #génération de mots
    Mstocks.append(PFin) #il faut prendre 
    return Mstocks #la matrice de la DERNIERE lettre et non celle qui correspond à la lettre à la place nombreDeLettres


#On essaye maintenant de combiner nos 2 méthodes les plus efficaces, on utilise le processus de chaine de markov
#avec des matrices prenant en compte les 2 lettres d'avant

def matriceProbaPosition2(file,placeLettre,mot,alphabet) :
    if placeLettre==0 or placeLettre==1 :
        M=matriceProbaPosition1(file, placeLettre,alphabet) #pour les deux premières lettres le code ne change pas, on peut
                                                               #on peut utiliser la fonction de matrice précédente
    else :
        M=np.zeros(55)
        comptageTot=0
        for ligne in file :
            if 0<placeLettre<len(ligne)-1 or placeLettre==-1 and 3<=len(ligne) :#je vérifie que je reste bien dans mon mot
                                    # la condition -1 sert lors de la matrice de la dernière lettre
                i=ligne[placeLettre-1] #lettre d'avant
                if i==mot[-1] : # je regarde si la lettre d'avant est égale à la dernière lettre actuelle de mon mot
                    j=ligne[placeLettre-2]
                    if j==mot[-2] : # de meme pour la lettre d'encore avant
                        lettre=ligne[placeLettre] 
                        k=alphabet[lettre] #numéro associé à la lettre analysée
                        M[k]+=1
                        comptageTot+=1
        if comptageTot !=0 :
            M=[M[i]/comptageTot for i in range(55)] #normalisation
    return M 


##################### GENERATION DE MOTS #####################

def genererMotMarkov1(nombreDeLettres,Mstocks):
    choix=np.random.choice(alphabetConverti,1,p=Mstocks[0]) #la 1ère lettre est hors de la boucle car 
    lettre=choix[0] #la Mstocks[0] n'a pas la meme dimension que les autres
    mot=lettre
    for i in range(1,nombreDeLettres):
        M=Mstocks[i] #matrice associée à la lettre en i-ème position
        ind=alphabet[lettre] #indice de la lettre précédente dans notre alphabet, nécessaire à notre matrice
        choix=np.random.choice(alphabetConverti,1,p=M[ind])
        lettreSuivante=choix[0]
        mot+=lettreSuivante # on ajoute la nouvelle lettre au mot
        lettre=lettreSuivante
    return mot
  


def genererMotMarkov2(file,nombreDeLettres,alphabet):
    lettre=np.random.choice(alphabetConverti,1,p=matriceProbaPosition2(file,0,[],alphabet))[0] 
    mot= lettre #1ère lettre
    ind=alphabet[lettre] #la 2ème lettre se génère comme dans genererMarkov1
    lettre=np.random.choice(alphabetConverti,1,p=matriceProbaPosition2(file,1,mot,alphabet)[ind])[0]
    mot+=lettre
    for i in range(2,nombreDeLettres-1 ) :
        M=matriceProbaPosition2(file,i,mot,alphabet) #On recalcule des matrices différentes ( en fait des listes) à chaque fois
        mot+=np.random.choice(alphabetConverti,1,p=M)[0]
    M=matriceProbaPosition2(file,-1,mot,alphabet) # on traite la dernière lettre à part
    mot+=np.random.choice(alphabetConverti,1,p=M)[0]
    return mot


##################### VISIONNER LES RESULTATS #####################

def visionMarkov1(n,nombremots,alphabet):
    sum=0
    Mstocks=matricesStockés(n,file,alphabet) #On génère toutes les matrices dont nous avons besoin,
                                             #ce sont les memes pour les mots de longueur identique
    print("génération de mots avec Markov1")
    for i in range(nombremots):
        t0=time.time()
        mot=genererMotMarkov1(n,Mstocks)
        tf=time.time()
        sum+=(tf-t0)
        print(mot)
    print("temps moyen de génération d'un mot:" +str(sum/nombremots)+"secondes" )

def visionMarkov2(n,nombremots,alphabet):
    sum=0
    print("génération de mots avec Markov2")
    for i in range(nombremots):
        t0=time.time()
        mot=genererMotMarkov2(file,n,alphabet)
        tf=time.time()
        sum+=(tf-t0)
        print(mot)
    print("temps moyen de génération d'un mot:" +str(sum/nombremots)+"secondes" )

visionMarkov1(10,10,alphabet)
visionMarkov2(10,10,alphabet)

génération de mots avec Markov1
pagancysse
fapliveras
cazisietit
pligelpens
encuctasis
adjoutease
exporiches
suisaneros
vifousente
tuberierai
temps moyen de génération d'un mot:0.001024341583251953secondes
génération de mots avec Markov2
oristatant
desalteras
insibation
mitromment
cesemaches
prepinants
matinciras
fistorames
cendalonir
budacerait
temps moyen de génération d'un mot:1.3859192371368407secondes


# Evaluation des programmes

L'idée ici est d'évaluer les programmes en fonction de la qualité de sortie des mots. 
### Cette évaluation sophistiquée demande quelques minutes à s'éxécuter

In [9]:
def decomposer3(l): #on entre la liste de mots
    motdecompose = []
    for k in range(len(l)):
        a=[]
        for j in range(len(l[k])-2):
            z = [l[k][j]+l[k][j+1]+l[k][j+2]]
            a+=z
            motdecompose.append(a)
    return motdecompose #la liste est décomposée en enchainements de 3

#Code qui permet de renvoyer un tableau renvoyant le nombre de fois où apparaissent les enchaînements de mots 
#en fonction de leur apparition  

def interdit(motdecompose,listeinterdit): #tu entres ta liste décomposée et la liste des enchainements de 2 ou 3 interdits.
    nbinterdit = []
    for k in range(len(motdecompose)):
        interdit = 0
        for j in range(len(motdecompose[k])):
            if motdecompose[k][j] in listeinterdit:
                interdit+=1
                nbinterdit.append(interdit)
    return nbinterdit #on obtient une liste des nombres d'apparition d'un enchainement 


#On cherche à avoir la matrice de probabilités d'apparition des trigrammes

a = decomposer3(file) #j'obtiens la liste de tous les trigrammes de mots.txt

#le probleme c'est que il me faut une liste à une dimension  alors que la liste a c'est [[1er mot décomposé],[2eme],...] donc il y a 2 dimensions
def aplatliste(L): #du coup on fait une fonction qui met les listes à 1 dimension
    R = []
    for elem in L:
        if isinstance(elem, (list, tuple)):
            R.extend(aplatliste(elem))
        else:
            R.append(elem)
    return R

b = aplatliste(a) #on met a à une dim du coup on a bien la liste de tous les trigrammes et on va pouvoir les compter avec l'outil Counter

In [10]:
from collections import Counter
counts = Counter(b) #j'obtiens ce que tu vois dans le fichier proba

d = dict(counts) #je crée le dictionnaire associé à counts qui a toutes les fréquences de chaque trigramme

#matrice des trigrammes

trigrammes = d.keys() #c'est la liste de tous les trigrammes présents dans mots.txt

Mat3 =[[[chr(97+i)+chr(97+j)+chr(97+k) for k in range(26)]for j in range(26)]for i in range(26)] #je crée la matrice qui répertorie les 26^3 combinaisons de trigrammes possibles

tot = len(b) #c'est le nombre total de trigrammes dans mots.txt

def M3(d):
    trigrammes = d.keys()
    v = np.zeros((26,26,26))
    for i in range(26):
        for j in range(26):
            for k in range(26):
                if Mat3[i][j][k] in trigrammes:
                    valeur = d.get(Mat3[i][j][k]) #permet d'avoir la valeur correspondante à un trigramme
                    v[i][j][k] = valeur/tot
    return v #j'obtiens la matrice de proba de toutes les combinaisons de trigramme

#Probabilité minimale d'apparition des trigrammes

def Min3(M):
    Min = 1
    for i in range(26):
        for j in range(26):
            for k in range(26):
                if 0 < M[i][j][k] < Min:
                    Min = M[i][j][k]
    return Min


#Une fois qu'on a la matrice on fait la fonction qui permet de trier les listes 

def jugement3(M,x,y):
    a=[]
    for i in range(26):
        for j in range(26):
            for k in range(26):
                if x < M[i][j][k] < y:
                    a.append(chr(97+i)+chr(97+j)+chr(97+k))
    return(a)

In [11]:
#Algorithme qui permet de transformer en liste les mots en sortie des fonctions génératrices de mots

def listevision2(n,nombremots,alphabet,genererMot2):
    y = []
    for i in range(nombremots):
        mot=genererMot2(n,alphabet)
        y.append(mot)
    return y

def listevision3(n,nombremots,alphabet,genererMot3):
    y = []
    for i in range(nombremots):
        mot=genererMot3(n,alphabet)
        y.append(mot)
    return y

def listevision2bis(n,nombremots,alphabet,genererMot2bis):
    y = []
    for i in range(nombremots):
        mot=genererMot2bis(n,alphabet)
        y.append(mot)
    return y

def listevision3bis(n,nombremots,alphabet,genererMot3bis):
    y = []
    for i in range(nombremots):
        mot=genererMot3bis(n,alphabet)
        y.append(mot)
    return y

def listevisionMarkov1(n,nombremots,alphabet):
    y = []
    Mstocks=matricesStockés(n, file, alphabet)
    for i in range(nombremots):
        mot=genererMotMarkov1(n,Mstocks)
        y.append(mot)
    return y

def listevisionMarkov2(n,nombremots,alphabet):
    y = []
    for i in range(nombremots):
        mot=genererMotMarkov2(file,n,alphabet)
        y.append(mot)
    return y

In [12]:
l1bis = jugement3(M3(d),0.001,1) #liste des trigrammes très fréquents
l2bis = jugement3(M3(d),0.0001,0.001) #liste des trigrammes fréquents
l3bis = jugement3(M3(d),0.00001,0.0001) #liste des trigrammes peu fréquents
l4bis = jugement3(M3(d),0.000001, 0.00001) #liste des trigrammes rares
l5bis = jugement3(M3(d),0.0000003, 0.000001) #liste des trigrammes très rares
#on part de la probabilité minimale et on monte jusqu'à 1 

#on crée des listes avec les mots en sortie pour pouvoir les décomposer et les analyser 
liste = listevision2(12,100, alphabet, genererMot2)
liste2 = listevision3(12,100,alphabet, genererMot3)
liste2bis = listevision2bis(12,100,alphabet,genererMot2bis)
liste3bis = listevision3bis(12, 100,alphabet,genererMot3bis)
listeMarkov1 = listevisionMarkov1(12, 100,alphabet)
listeMarkov2 = listevisionMarkov2(12, 100,alphabet)

#On affiche les listes qui comptabilisent le nombre d'apparition des enchaînements en fonction des seuils de probabilités choisis
#On fait ça pour les 6 programmes qui génèrent des mots


In [13]:
#système de notation
#L'idée ici c'est de faire un algorithme qui ajoute des points avec un coefficient en fonction des seuils auxquels appartiennent
#les trigrammes qui composent les mots pour arriver à une note sur 10 et estimer quel est le meilleur programme
import math as m

def notation(l):
    n=1
    s=0
    tot=0
    for i in range(3): #points positifs pour les trigrammes fréquents à très fréquents
        a=sum(l[i])
        s+=a*n #on multiplie la note par son coefficient n
        tot+=a
        n-=0.25
    n=0.5
    for i in range(3,5): #points négatifs pour les trigammes rares et très rares
        a=sum(l[i])
        s-=a*n #on multiplie la note par son coefficient n
        tot+=a
        n+=0.25
    return (s/tot)*10 #on convertit la note sur 10 

    

#on crée des listes de listes en fonction de chaque programme pour pouvoir sommer ensuite les notes

l1 = [interdit(decomposer3(liste),l1bis), interdit(decomposer3(liste),l2bis), interdit(decomposer3(liste),l3bis), interdit(decomposer3(liste),l4bis), interdit(decomposer3(liste),l5bis)]

l2 = [interdit(decomposer3(liste2),l1bis), interdit(decomposer3(liste2),l2bis), interdit(decomposer3(liste2),l3bis), interdit(decomposer3(liste2),l4bis), interdit(decomposer3(liste2),l5bis)]

l3 = [interdit(decomposer3(liste2bis),l1bis), interdit(decomposer3(liste2bis),l2bis), interdit(decomposer3(liste2bis),l3bis), interdit(decomposer3(liste2bis),l4bis), interdit(decomposer3(liste2bis),l5bis)]

l4 = [interdit(decomposer3(liste3bis),l1bis), interdit(decomposer3(liste3bis),l2bis), interdit(decomposer3(liste3bis),l3bis), interdit(decomposer3(liste3bis),l4bis), interdit(decomposer3(liste3bis),l5bis)]

l5 = [interdit(decomposer3(listeMarkov1),l1bis), interdit(decomposer3(listeMarkov1),l2bis), interdit(decomposer3(listeMarkov1),l3bis), interdit(decomposer3(listeMarkov1),l4bis), interdit(decomposer3(listeMarkov1),l5bis)]

l6 = [interdit(decomposer3(listeMarkov2),l1bis), interdit(decomposer3(listeMarkov2),l2bis), interdit(decomposer3(listeMarkov2),l3bis), interdit(decomposer3(listeMarkov2),l4bis), interdit(decomposer3(listeMarkov2),l5bis)]

print("note sans Markov, enchaînements de deux lettres, sans contrôle: ", notation(l1), "/10")
print("note sans Markov, enchaînements de trois lettres, sans contrôle: ", notation(l2), "/10")
print("note sans Markov, enchaînements de deux lettres, avec contrôle: ", notation(l3), "/10")
print("note sans Markov, enchaînements de trois lettres, avec contrôle: ", notation(l4), "/10")
print("note avec Markov, enchaînements de deux lettres: ", notation(l5), "/10")
print("note avec Markov, enchaînements de trois lettres: ", notation(l6), "/10")

note sans Markov, enchaînements de deux lettres, sans contrôle:  7.99923925446938 /10
note sans Markov, enchaînements de trois lettres, sans contrôle:  8.767994643454971 /10
note sans Markov, enchaînements de deux lettres, avec contrôle:  8.031311154598825 /10
note sans Markov, enchaînements de trois lettres, avec contrôle:  8.65244708994709 /10
note avec Markov, enchaînements de deux lettres:  8.248299319727892 /10
note avec Markov, enchaînements de trois lettres:  8.945177505497957 /10
